In [19]:
import pandas as pd
import pybedtools
import numpy as np
import os

In [2]:
input_path_minus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_1hr_S2_CPD_1bp_sorted_minus.bed"
input_path_plus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_1hr_S2_CPD_1bp_sorted_plus.bed"
input_path_noUV_minus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_noUV_S1_CPD_1bp_sorted_minus.bed"
input_path_noUV_plus = "/data/private/pdutta/Collab_data/Jim_work/CPD-seq/04-damage-1bp/Tissue_noUV_S1_CPD_1bp_sorted_plus.bed"

In [18]:
wgs_data_folder = "/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/"

In [3]:
df_plus =pd.read_csv(input_path_plus, sep="\t", header=None)
df_minus =pd.read_csv(input_path_minus, sep="\t", header=None)
df_noUV_minus =pd.read_csv(input_path_noUV_minus, sep="\t", header=None)
df_noUV_plus =pd.read_csv(input_path_noUV_plus, sep="\t", header=None)

In [4]:
print(df_plus.shape, df_noUV_plus.shape)

(99495320, 5) (15069500, 5)


In [6]:
df_plus = df_plus.drop_duplicates()
df_plus

,0,1,2,3,4
0,chr1,3050029,3050030,+,AA
1,chr1,3050030,3050031,+,AA
2,chr1,3050054,3050055,+,GG
3,chr1,3050055,3050056,+,GG
4,chr1,3050076,3050077,+,AG
...,...,...,...,...,...
99495301,pUC19,2626,2627,+,GA
99495306,pUC19,2634,2635,+,AA
99495310,pUC19,2635,2636,+,AA
99495314,pUC19,2635,2636,+,AG


In [7]:
df_noUV_plus = df_noUV_plus.drop_duplicates()
df_noUV_plus

,0,1,2,3,4
0,chr1,3050021,3050022,+,GA
1,chr1,3050022,3050023,+,GA
2,chr1,3050413,3050414,+,AG
3,chr1,3050414,3050415,+,AG
4,chr1,3050720,3050721,+,GG
...,...,...,...,...,...
15069474,pUC19,2626,2627,+,GA
15069478,pUC19,2634,2635,+,AA
15069488,pUC19,2635,2636,+,AA
15069498,pUC19,2635,2636,+,AG


In [8]:
df_merged = pd.merge(df_plus, df_noUV_plus, how="outer", indicator=True)
df_merged

,0,1,2,3,4,_merge
0,chr1,3050029,3050030,+,AA,left_only
1,chr1,3050030,3050031,+,AA,left_only
2,chr1,3050054,3050055,+,GG,left_only
3,chr1,3050055,3050056,+,GG,left_only
4,chr1,3050076,3050077,+,AG,left_only
...,...,...,...,...,...,...
103404139,chrY_JH584303v1_random,155795,155796,+,AG,right_only
103404140,chrY_JH584303v1_random,156895,156896,+,GA,right_only
103404141,chrY_JH584303v1_random,156896,156897,+,GA,right_only
103404142,pUC19,1616,1617,+,GA,right_only


In [10]:
df_merged['_merge'].unique()

['left_only', 'both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [11]:
only_in_df_plus = df_merged[df_merged['_merge'] == 'left_only'].drop('_merge', axis=1)
only_in_df_plus

,0,1,2,3,4
0,chr1,3050029,3050030,+,AA
1,chr1,3050030,3050031,+,AA
2,chr1,3050054,3050055,+,GG
3,chr1,3050055,3050056,+,GG
4,chr1,3050076,3050077,+,AG
...,...,...,...,...,...
90994605,chrY_JH584303v1_random,157723,157724,+,AG
90994606,chrY_JH584303v1_random,157983,157984,+,GA
90994607,chrY_JH584303v1_random,157984,157985,+,GA
90994608,chrY_JH584303v1_random,158010,158011,+,GG


In [13]:
only_in_noUV_df_plus = df_merged[df_merged['_merge'] == 'right_only'].drop('_merge', axis=1)
only_in_noUV_df_plus

,0,1,2,3,4
90995852,chr1,3050021,3050022,+,GA
90995853,chr1,3050022,3050023,+,GA
90995854,chr1,3050413,3050414,+,AG
90995855,chr1,3050414,3050415,+,AG
90995856,chr1,3050721,3050722,+,GG
...,...,...,...,...,...
103404139,chrY_JH584303v1_random,155795,155796,+,AG
103404140,chrY_JH584303v1_random,156895,156896,+,GA
103404141,chrY_JH584303v1_random,156896,156897,+,GA
103404142,pUC19,1616,1617,+,GA


In [12]:
both_in_df = df_merged[df_merged['_merge'] == 'both'].drop('_merge', axis=1)
both_in_df

,0,1,2,3,4
53,chr1,3050720,3050721,+,GG
60,chr1,3050881,3050882,+,GG
84,chr1,3051229,3051230,+,AA
108,chr1,3051533,3051534,+,GG
109,chr1,3051534,3051535,+,GG
...,...,...,...,...,...
90995847,pUC19,2626,2627,+,GA
90995848,pUC19,2634,2635,+,AA
90995849,pUC19,2635,2636,+,AA
90995850,pUC19,2635,2636,+,AG


In [16]:
only_in_noUV_df_plus[0].unique()

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr1_GL456210v1_random',
       'chr1_GL456211v1_random', 'chr1_GL456212v1_random',
       'chr1_GL456221v1_random', 'chr1_GL456239v1_random',
       'chr1_MU069434v1_random', 'chr2', 'chr3', 'chr4',
       'chr4_JH584295v1_random', 'chr5', 'chr5_GL456354v1_random',
       'chr5_JH584296v1_random', 'chr5_JH584297v1_random',
       'chr5_JH584298v1_random', 'chr5_JH584299v1_random', 'chr6', 'chr7',
       'chr7_GL456219v1_random', 'chr8', 'chr9', 'chrM',
       'chrUn_GL456359v1', 'chrUn_GL456360v1', 'chrUn_GL456366v1',
       'chrUn_GL456367v1', 'chrUn_GL456368v1', 'chrUn_GL456370v1',
       'chrUn_GL456372v1', 'chrUn_GL456378v1', 'chrUn_GL456379v1',
       'chrUn_GL456381v1', 'chrUn_GL456382v1', 'chrUn_GL456383v1',
       'chrUn_GL456385v1', 'chrUn_GL456387v1', 'chrUn_GL456389v1',
       'chrUn_GL456390v1', 'chrUn_GL456392v1', 'chrUn_GL456394v1',
       'chrUn_GL456396v1', 

In [17]:
only_in_noUV_df_plus.groupby([0]).size()

0
chr1                      912599
chr10                     604646
chr11                     642427
chr12                     564011
chr13                     570294
                           ...  
chrY_JH584300v1_random       480
chrY_JH584301v1_random       530
chrY_JH584302v1_random       362
chrY_JH584303v1_random       369
pUC19                          2
Length: 62, dtype: int64

# WGS data (SNV)

In [42]:
combined_df = pd.DataFrame()
for filename in os.listdir(wgs_data_folder):
    # Check if the file is a txt file
    if filename.endswith('.txt'):
        # Construct the full filepath
        filepath = os.path.join(wgs_data_folder, filename)
        print(filepath)
        
        # Read the file into a DataFrame
        temp_df = pd.read_csv(filepath, delimiter='\t', header=None)  # Assuming tab-delimited, change as needed
        
        
        # Append the data to the combined DataFrame
        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

# Show the combined DataFrame

/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/15_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/Y_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/10_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/2_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/18_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/19_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/13_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/1_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/3_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/5_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/8_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/12_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/6_seqinfo.txt
/data/private/pdutta/Collab_data/Jim_work/Mouse_WGS/SNV/9_seqinfo.txt
/data/private/

In [43]:
combined_df

,0,1,2,3,4
0,S319_final_vep_snpfiltered,15,3066686,N:AC[C>T]GT,-1
1,S319_final_vep_snpfiltered,15,3391439,T:GG[C>T]GG,1
2,S319_final_vep_snpfiltered,15,3911756,N:TC[C>T]TT,1
3,S319_final_vep_snpfiltered,15,3930627,N:AT[C>T]CC,1
4,S319_final_vep_snpfiltered,15,4574999,T:CC[C>T]TT,-1
...,...,...,...,...,...
828281,S346_final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
828282,S346_final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
828283,S346_final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
828284,S346_final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [44]:
combined_df[['ID', '0']] = combined_df[0].str.split('_', 1, expand=True)

# Drop the original column
combined_df.drop(columns=[0], inplace=True)

# Reorder the columns to make new_col1 and new_col2 the first and second columns
combined_df = combined_df[['ID', '0'] + [col for col in combined_df.columns if col not in ['ID', '0']]]
combined_df['ID'] = combined_df['ID'].str.replace('S','').astype(int)
combined_df

/tmp/ipykernel_33171/4152682433.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  combined_df[['ID', '0']] = combined_df[0].str.split('_', 1, expand=True)


,ID,0,1,2,3,4
0,319,final_vep_snpfiltered,15,3066686,N:AC[C>T]GT,-1
1,319,final_vep_snpfiltered,15,3391439,T:GG[C>T]GG,1
2,319,final_vep_snpfiltered,15,3911756,N:TC[C>T]TT,1
3,319,final_vep_snpfiltered,15,3930627,N:AT[C>T]CC,1
4,319,final_vep_snpfiltered,15,4574999,T:CC[C>T]TT,-1
...,...,...,...,...,...,...
828281,346,final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
828282,346,final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
828283,346,final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
828284,346,final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [49]:
df_interest = df_interest = combined_df[(combined_df["ID"] >= 332) & (combined_df["ID"] <= 346)].reset_index(drop=True)
df_interest

,ID,0,1,2,3,4
0,334,final_vep_snpfiltered,15,3058069,N:CC[C>T]TA,-1
1,334,final_vep_snpfiltered,15,3063770,N:AA[T>A]AT,-1
2,334,final_vep_snpfiltered,15,3080731,N:GT[C>T]TG,1
3,334,final_vep_snpfiltered,15,3094450,N:AG[T>G]TT,1
4,334,final_vep_snpfiltered,15,3109676,N:AC[C>T]CA,1
...,...,...,...,...,...,...
421491,346,final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1
421492,346,final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1
421493,346,final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1
421494,346,final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1


In [50]:
df_interest.groupby([1]).size()

1
1     33112
2     27751
3     28849
4     25487
5     27205
6     25137
7     18750
8     20731
9     19970
10    20990
11    18882
12    19775
13    19693
14    19182
15    17638
16    16528
17    14918
18    12281
19     7296
X     27320
Y         1
dtype: int64

In [51]:
df_interest[3]

0         N:CC[C>T]TA
1         N:AA[T>A]AT
2         N:GT[C>T]TG
3         N:AG[T>G]TT
4         N:AC[C>T]CA
             ...     
421491    N:CT[C>T]TT
421492    N:CT[C>T]TT
421493    N:AA[C>T]TT
421494    N:TC[C>T]TT
421495    U:TT[C>T]TG
Name: 3, Length: 421496, dtype: object

In [52]:
df_interest['type'] = df_interest[3].apply(lambda x: x.split(":")[0])
df_interest

,ID,0,1,2,3,4,type
0,334,final_vep_snpfiltered,15,3058069,N:CC[C>T]TA,-1,N
1,334,final_vep_snpfiltered,15,3063770,N:AA[T>A]AT,-1,N
2,334,final_vep_snpfiltered,15,3080731,N:GT[C>T]TG,1,N
3,334,final_vep_snpfiltered,15,3094450,N:AG[T>G]TT,1,N
4,334,final_vep_snpfiltered,15,3109676,N:AC[C>T]CA,1,N
...,...,...,...,...,...,...,...
421491,346,final_vep_snpfiltered,X,168111504,N:CT[C>T]TT,1,N
421492,346,final_vep_snpfiltered,X,168232166,N:CT[C>T]TT,1,N
421493,346,final_vep_snpfiltered,X,168273413,N:AA[C>T]TT,1,N
421494,346,final_vep_snpfiltered,X,168326651,N:TC[C>T]TT,-1,N


In [53]:
df_interest['type'].unique()

array(['N', 'T', 'U', 'B'], dtype=object)

## Found four type of tags  
   * __N__: This might stand for "Normal," perhaps indicating that the sequence comes from normal tissue or represents a reference sequence.
   * __T__: This could stand for "Tumor," possibly indicating that the sequence comes from tumor tissue, especially if you're working on cancer genomics data.
   * __U__: This could stand for "Unknown," perhaps indicating that the origin of the sequence or the quality of the data is uncertain.
   * __B__: This is not standard in genomics as far as my training data goes. It could be specific to the particular dataset or pipeline you're working with. It might stand for something like "Baseline" or "Both" (if it's a combined analysis of tumor and normal, for example), but that's purely speculative.

In [56]:
df_interest.groupby('type').size()

type
B       940
N    277823
T     56554
U     86179
dtype: int64

In [59]:
df_interest_bed = pd.DataFrame()
df_interest_bed['chr'] = 'chr'+ df_interest[1].astype(str)
df_interest_bed['start'] = df_interest[2]-1
df_interest_bed['end'] = df_interest[2]
df_interest_bed

,chr,start,end
0,chr15,3058068,3058069
1,chr15,3063769,3063770
2,chr15,3080730,3080731
3,chr15,3094449,3094450
4,chr15,3109675,3109676
...,...,...,...
421491,chrX,168111503,168111504
421492,chrX,168232165,168232166
421493,chrX,168273412,168273413
421494,chrX,168326650,168326651


In [62]:
chromosomes_to_keep = ['chr' + str(i) for i in range(1, 23)] + ['chrX', 'chrY']
df_filtered_noUV_plus_only = only_in_noUV_df_plus[only_in_noUV_df_plus[0].isin(chromosomes_to_keep)].reset_index(drop=True)
df_filtered_noUV_plus_only

,0,1,2,3,4
0,chr1,3050021,3050022,+,GA
1,chr1,3050022,3050023,+,GA
2,chr1,3050413,3050414,+,AG
3,chr1,3050414,3050415,+,AG
4,chr1,3050721,3050722,+,GG
...,...,...,...,...,...
12384742,chrY,90855666,90855667,+,GA
12384743,chrY,90855677,90855678,+,GG
12384744,chrY,90855678,90855679,+,GG
12384745,chrY,90855830,90855831,+,AA


In [65]:
df_filtered_noUV_plus_only.head(20)

,0,1,2,3,4
0,chr1,3050021,3050022,+,GA
1,chr1,3050022,3050023,+,GA
2,chr1,3050413,3050414,+,AG
3,chr1,3050414,3050415,+,AG
4,chr1,3050721,3050722,+,GG
5,chr1,3050756,3050757,+,GA
6,chr1,3050757,3050758,+,GA
7,chr1,3050807,3050808,+,GA
8,chr1,3050808,3050809,+,GA
9,chr1,3050842,3050843,+,AA


## Finding Intersection using `PyBedtools`

In [66]:
df_interest_SNV_bed = pybedtools.BedTool.from_dataframe(df_interest_bed)
df_noUV_plus_only_bed = pybedtools.BedTool.from_dataframe(df_filtered_noUV_plus_only)
intersect_SNV_CPD = df_interest_SNV_bed.intersect(df_noUV_plus_only_bed, wa=True, wb=True)
intersect_SNV_CPD.to_dataframe()

,chrom,start,end,name,score,strand,thickStart,thickEnd
0,chr15,3895294,3895295,chr15,3895294,3895295,+,GG
1,chr15,5579415,5579416,chr15,5579415,5579416,+,AG
2,chr15,6191775,6191776,chr15,6191775,6191776,+,GA
3,chr15,6207261,6207262,chr15,6207261,6207262,+,GA
4,chr15,9691374,9691375,chr15,9691374,9691375,+,GG
...,...,...,...,...,...,...,...,...
3501,chrX,147720411,147720412,chrX,147720411,147720412,+,GG
3502,chrX,39568936,39568937,chrX,39568936,39568937,+,GA
3503,chrX,88746130,88746131,chrX,88746130,88746131,+,GG
3504,chrX,104886995,104886996,chrX,104886995,104886996,+,GG
